In [1]:
using Contour
using JLD
using ColorSchemes
using Statistics
using DelimitedFiles
using Distributions
using StructArrays
using StatsBase
using Interpolations
using QuadGK
using Optim
using Roots
using HypothesisTests
using KernelDensity
using QuadGK

using Revise
using GravityToolsLight
using PyPlot
pygui(true)

true

In [ ]:
basic_settings = BasicTempoSettings(
    work_dir = "/Users/abatrakov/Documents/Work/PhD/projects/J1141-6545/full_WN",
    version = Tempo2(),
    par_file_init = "J1141-6545_until_2018_DDSTG.par",
    tim_file = "J1141-6545_until_2018.tim",
    flags = "-nobs 34000 -newpar -writeres -residuals",
    tparams = [TP("EOS", "BSk22"), TP("COMP_TYPE", "WD"), TP("ALPHA0", 0.0), TP("BETA0", 0.0), TP("NITS", 3)],
    keys = BasicTempoKeys(silent=true, print_output=true, save_internal_iterations=true, fit_EFACs_EQUADs=false)
    )
    
results_basic = run_tempo_basic(basic_settings)

In [ ]:
basic_settings = BasicTempoSettings(
    work_dir = "/Users/abatrakov/Documents/Work/PhD/projects/J1141-6545/full_data",
    version = Tempo2(),
    par_file_init = "ddgr_latest.par",
    tim_file = "J1141-6545_pn.tim",
    flags = "-nobs 23000 -newpar -writeres -residuals",
    tparams = [TP("NITS", 3), TP("XPBDOT", flag=1)],
    keys = BasicTempoKeys(silent=true, print_output=true, save_internal_iterations=true, fit_EFACs_EQUADs=false)
    )
    
results_basic = run_tempo_basic(basic_settings)

In [4]:
#------------------------------------------------------------------------------------- 
# WE HAVE PRIOR INFORMATION ABOUT XPBDOT
# use sweep or global iterations over XPBDOT to check likelihood
#-------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------
# FIRSTLY WE FIT FOR XPBDOT IN TEMPO
#-------------------------------------------------------------------------------------
basic_settings = BasicTempoSettings(
    work_dir = "/Users/abatrakov/Documents/Work/PhD/projects/J1141-6545/full_data",
    version = Tempo2(),
    par_file_init = "DDSTG_init.par",
    tim_file = "J1141-6545_pn.tim",
    flags = "-nobs 23000 -newpar -writeres -residuals",
    tparams = [TP("EOS", "BSk22"), TP("COMP_TYPE", "WD"), TP("ALPHA0", 0.0), TP("BETA0", 0.0), TP("NITS", 1), TP("XPBDOT", -8.3e-14, flag=0)],
    keys = BasicTempoKeys(silent=true, print_output=true, save_internal_iterations=true, fit_EFACs_EQUADs=false)
    )
    
results_basic = run_tempo_basic(basic_settings)

In [3]:

# BEST FIT 
chisqr_min = results_basic.last_internal_iteration.result.chisqr
xpbdot_mu = results_basic.last_internal_iteration.result.XPBDOT.post_fit
xpbdot_sigma = results_basic.last_internal_iteration.result.XPBDOT.uncertainty

# WE OBTAINED MU AND SIGMA FOR NORMAL LIKELIHOOD
# chisqr_min, xpbdot_mu, xpbdot_sigma = 20914.19, -8.29481968690372e-15,  4.7427e-15
# BUILD NORMAL LIKELIHOOD
xpbdot_normal_likelihood(xpbdot) = pdf(Normal(xpbdot_mu, xpbdot_sigma), xpbdot)
# BUILD NORMAL LIKELIHOOD IN CHISQR REPRESENTATION
delta_chisqr_normal_likelihood(xpbdot) = -2 * log(xpbdot_normal_likelihood(xpbdot)) + 2 * log(xpbdot_normal_likelihood(xpbdot_mu))

General Tempo Result:
    Internal iteration:
        Detailed Tempo Output Result:
            Basic Tempo Output Result:
                Fit Chisq: 22260.0
                Chisqr: 22257.74
                Number of Points in Fit: 22285
                Number of Fit Parameters: 192
                RMS Pre-fit Residual (us): 9877.174
                Nfree: 22259
                Offset: (0.00583175, 0.00789693)
                Offset E * Sqrt(n): 1.17887
                Pre/Post: 1.00009
                RMS Post-fit Residual (us): 9876.284
                Chisqr/Nfree: 0.999944
            Fit Parameters:
                RAJ  Pre-fit: 3.05919880576274  Post-fit: 3.05919880576129  Unc: 1.5761e-7  Diff: -1.4495e-12  fit: true
                DECJ  Pre-fit: -1.14764679499624  Post-fit: -1.14764679499371  Unc: 6.3664e-8  Diff: 2.5231e-12  fit: true
                F0  Pre-fit: 2.53872304037579  Post-fit: 2.5387230403758  Unc: 2.5385e-10  Diff: 8.4377e-15  fit: true
                F1  Pre-f